In [1]:
import torch, textwrap
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

/home/kilnaar/anaconda3/envs/ai574-pocs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load model

In [34]:
MODEL_ID = "johnsnowlabs/JSL-MedLlama-3-8B-v2.0"
DEVICE   = "cuda" if torch.cuda.is_available() else "CPU"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, load_in_8bit = True, device_map = "auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.68s/it]
We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


# Some medical text

In [35]:
medical_text = """
Non-steroidal anti-inflammatory drugs are not only potent analgesics and antipyretics but also nephrotoxins, and may cause 
electrolyte disarray. In addition to the commonly expected effects, including hyperkalemia, hyponatremia, acute renal injury, 
renal cortical necrosis, and volume retention, glomerular disease with or without nephrotic syndrome or nephritis can occur as 
well including after years of seemingly safe administration. Minimal change disease, secondary membranous glomerulonephritis, 
and acute interstitial nephritis are all reported glomerular lesions seen with non-steroidal anti-inflammatory use. We report a 
patient who used non-steroidal anti-inflammatory drugs for years without diabetes, chronic kidney disease, or proteinuria; he 
then developed severe nephrotic range proteinuria with 7 g of daily urinary protein excretion. Renal biopsy showed minimal 
change nephropathy, a likely secondary membranous glomerulonephritis, and acute interstitial nephritis present simultaneously
in one biopsy. 

"""

In [83]:
messages = [
    {
        "role": "user",
        "content": textwrap.dedent(f"""
            Below is an abstract from a medical paper.

            ```text
            {medical_text.strip()}
            ```

            **Task:** Produce a ≤80-word summary **and end with a full stop (.) when you are done.**
            Use clear, professional medical language.
            Don't include a greeting or introduction.
        """),
    }
]

# Inference

In [87]:
prompt_ids = tokenizer.apply_chat_template(
    messages, add_generation_prompt = True, return_tensors="pt").to(model.device)

gen_cfg = GenerationConfig(
    max_new_tokens = 128,
    temperature = 0.1,
    top_p = 0.9,
    repetition_penalty = 1.1,
    do_sample = True,
    no_repeat_ngram_size = 6,
)

with torch.no_grad():
    generated = model.generate(prompt_ids, generation_config = gen_cfg, return_dict_in_generate = False)

summary = tokenizer.decode(generated[0], skip_special_tokens = True)
response_text = textwrap.fill(summary, 90).split('im_start|>')[-1]
print(response_text)

user  Here is the text: Non-
steroidal anti inflammatory drugs (NSAIDs) have multiple effects on the body beyond their
role as pain relievers and fever reducers. They can cause kidney damage, electrolyte
imbalances, and in rare cases, glomerular diseases such as minimal change disease,
secondary membr
